[Spaceship Titanic](https://www.kaggle.com/code/odins0n/spaceship-titanic-eda-27-different-models#Modeling-)
어떤 승객이 대체 차원으로 전송될지 예측해보자.


>https://www.kaggle.com/competitions/spaceship-titanic/overview

# Introduction

**시놉시스**

우주의 신비를 풀기 위해 데이터 과학 기술이 필요한 2912년에 온 것을 환영한다.   
우리는 4광년 떨어진 곳에서 전송을 받았고 상황이 좋아 보이지 않는다.  

**Spaceship Titanic**은 한 달 전에 발사된 성간 여객선으로 약 13,000명의 승객의 탑승한 이 선박은 우리 태양계에서 인근 별을 공전하는  
새로 거주 가능한 세 개의 외계 행성으로 이민자들을 수송하는 처녀 항해를 시작했다.  

알파 센타우리를 첫 번째 목적지인 55 칸크리 E로 도는 동안, 타이타닉근 먼지 구름 속 숨겨진 시공간의 이상 물체와 충돌했다.  
슬프게도 선박은 1000년 전 이름을 딴 것과 비슷한 운명을 마주했다.  
배는 손상되지 않았지만, 승객의 거의 절반이 대체 차원으로 전송되었다. 

승무원을 구조하고 잃어버린 승객을 회수하기 위해, 우주선의 손상된 컴퓨터 시스템에서 회수된 기록을 사용해  
어떤 승객이 대체 차원으로 전송되었는지 예측해야 한다. 

그들을 구하고 역사를 바꾸도록 도와달라!

**평가** 
제출한 파일은 정확한 예측 라벨의 비율인 **분류 정확도**에 따라 평가된다. (classification accuracy)

# Imports

In [1]:
# from IPython.display import clear_output
# !pip3 install -U lazypredict ##lazypredict?
# !pip3 install -U pandas

# clear_output()

In [2]:
# !pip3 install scikit-learn==0.23.1
# !sklearn --version
# !pip3 show scikit-learn  # 현재 버전 확인

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split


from lightgbm import LGBMClassifier
import lazypredict
from lazypredict.Supervised import LazyClassifier

import time
import warnings
warnings.filterwarnings('ignore')

/opt/homebrew/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


> [LazyClassifier 참고](https://bourbonkk.tistory.com/113)  
> [from lazypredict.Surpervised import LazyClassifier 오류 해결](https://github.com/shankarpandala/lazypredict/issues/325) 
>> 1. vi Superviesd.py 내부 패키지 변경.
>> 2. .pop() 이후 조건문 추가.

In [4]:
from sklearn.linear_model import TheilSenRegressor

# Data Loading and Preparation

In [5]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
submission = pd.read_csv('sample_submission.csv')

Random_state = 12
Folds = 5
Strategy='median'

FileNotFoundError: [Errno 2] No such file or directory: './train.csv'

**Columns 설명**
* **PassengerID**
 - 각 승객을 위한 독특한 ID. 각 ID는 gggg_pp 양식을 취하며, 여기서 gggg는 승객이 여행하는 그룹을 나타내고 pp는 그룹 내의 번호입니다.    그룹의 사람들은 종종 가족 구성원이지만, 항상 그런 것은 아니다.
* **HomePlanet**
 - 승객이 떠난 행성, 일반적으로 영주권 행성.
* **CryoSleep**
 - 항해 중 승객이 가사 상태에 놓이도록 선택했는지 여부를 나타낸다.  
   극 저온 수면 상태의 승객들을 객실에 갇혀 있다.
* **Cabin**
 - 승객이 머무는 승객 번호이다.  
 - 폼 데크/넘버/사이드로 지정한다.  
 (여기서 측변은 좌현에 대해 P, 우현에 대해 S가 될 수 있다.)
* **Destination**
 - 승객이 하선할 행성
* **Age**
 - 승객의 연령
* **VIP**
 - 승객이 항해 중 특별 VIP 서비스에 대한 비용을 지불했는지 여부
* **RoomService, Food Court, ShoppingMall, Spa, VRDeck**
 - 승객이 지불한 타이타닉 럭셔리 어메니티 금액
* **Name**
 - 승객의 성과 이름
* **Transported**
 - 승객이 다른 차원으로 이송되었는지 여부. 
 - 이것은 당신이 예측하려고 하는 목표, 열입니다.

## Train data 탐색
 - 훈련 데이터는 14개 열과 8693개의 행으로 구성되어 있다.
 - 훈련 데이터는 2234개의 결측값을 포함한 119378개의 관찰값으로 구성되어 있다.
 - 12개의 변수 모두 누락된 값이 있고, **CryoSleep**변수의 누락값이 가장 많다.(217개)
 - **<u>Transported</u>**는 훈련 데이터 세트에서만 사용할 수 있는 Target 변수이다.

### Train Data 확인

In [ ]:
train.head()

In [ ]:
print(f'\033[94mNumber of rows in train data: {train.shape[0]}')
print(f'\033[94mNumber of columns in train data: {train.shape[1]}')
print(f'\033[94mNumber of values in train data: {train.count().sum()}')
print(f'\033[94mNumber of missing values in train data: {sum(train.isna().sum())}')

### Columns에서 누락된 값 확인

In [ ]:
print(f'\033[94m')
print(train.isnull().sum().sort_values(ascending=False))

### training data 기초 통계
아래는 수, 평균, 표준 편차, 최소값, 1사분위수, 중앙값, 3사분위수 및 최댓값에 대한 정보를 포함하는 각 변수의 기초 통계이다. 

In [ ]:
train.describe().T

## Test data 탐색

* 총 13개의 열과 4277개의 행으로 구성되어 있다.
* test 데이터는 1117개의 결측값을 포함한 54484개의 데이터로 구성되어 있다.
* 12개의 데이터는 모두 결측값을 가지고 있으며 **FoodCourt**데이터의 결측값이 가장 많은 것을 알 수 있다.(106개)

In [ ]:
test.head()

In [ ]:
print(f'\033[94mNUmber of row in test data: {test.shape[0]}')
print(f'\033[94mNUmber of columns in test data: {test.shape[1]}')
print(f'\033[94mNUmber of values in test data: {test.count().sum()}')
print(f'\033[94mNUmber of rows with missing in test data: {sum(test.isna().sum())}')

### column별 결측값

In [ ]:
print(f'\033[94m')
print((test.isna().sum().sort_values(ascending=False)))

### test data 기초 통계
아래는 수, 평균, 표준 편차, 최소값, 1사분위수, 중앙값, 3사분위수 및 최댓값에 대한 정보를 포함하는 각 변수의 기초 통계이다. 

In [ ]:
test.describe().T

## Submission File

In [ ]:
submission.head()

# EDA

## Data 전체보기

In [ ]:
train.drop(['PassengerId'], axis=1, inplace=True)
test.drop(['PassengerId'], axis=1,inplace=True)

Target = 'Transported'
Features = [col for col in train.columns if col != Target]
Random_state=12

In [ ]:
train.head()

In [ ]:
train.iloc[:,:-1].describe().T.sort_values(by='std', ascending=False)\
.style.background_gradient(cmap='GnBu')\
.bar(subset=['max'], color='#BB0000').bar(subset=['mean'], color='green')


## 결측값 분포
* 행에서 누락된 값의 최댓값은 3이고 최솟값은 누락되지 않았다.
* 흥미롭게도, 결측값 분포(행 기준)는 train, test 세트 가 동일하다.
* 약 76%의 관찰값은 결측값이 없는 것을 알 수 있다.(행 기준)
* 결측값이 1-3개인 관측치(행 기준)의 나머지 24%

## column 기준 결측치 분포

In [ ]:
test_null=pd.DataFrame(test.isna().sum())
test_null=test_null.sort_values(by=0, ascending=False)

train_null=pd.DataFrame(train.isna().sum())
train_null=train_null.sort_values(by=0, ascending=False)[:-1]

In [ ]:
fig = make_subplots(rows=1, cols=2, column_titles=['Train Data', 'Test Data'], x_title='Missing Values')

## train columns 표시
fig.add_trace(go.Bar(x=train_null[0], y=train_null.index, orientation='h',
                     marker=dict(color=[n for n in range(12)],
                     line_color='rgb(0,0,0)',
                     line_width=2,
                     coloraxis='coloraxis')), 1,1)

## test columns 표시
fig.add_trace(go.Bar(x=test_null[0], y=test_null.index, orientation='h',
                    marker=dict(color=[n for n in range(12)],
                                line_color='rgb(0,0,0)',
                                line_width=2,
                                coloraxis='coloraxis')),1,2)

fig.update_layout(showlegend=False, title_text='Column wise Null Value Distribution', title_x=0.5)

## row 기준 결측치 분포

In [ ]:
missing_train_row=train.isna().sum(axis=1)
missing_train_row=pd.DataFrame(missing_train_row.value_counts()/train.shape[0]).reset_index()

missing_test_row=test.isna().sum(axis=1)
missing_test_row=pd.DataFrame(missing_test_row.value_counts()/test.shape[0]).reset_index()

missing_train_row.columns=['no','count']
missing_test_row.columns=['no','count']

missing_train_row['count'] = missing_train_row['count']*100
missing_test_row['count'] = missing_test_row['count']*100

In [ ]:
fig = make_subplots(rows=1, cols=2, column_titles=['Train Data','Test Data'], x_title='Missing Values')

fig.add_trace(go.Bar(x=missing_train_row['no'], y=missing_train_row['count'],
                    marker=dict(color=[n for n in range(4)],
                               line_color='rgb(0,0,0)',
                               line_width=3,
                               coloraxis='coloraxis')),1,1)

fig.add_trace(go.Bar(x=missing_test_row['no'], y=missing_test_row['count'],
                    marker=dict(color=[n for n in range(4)],
                               line_color='rgb(0,0,0)',
                               line_width=3,
                               coloraxis='coloraxis')),1,2)

fig.update_layout(showlegend=False, title_text='Row wise Null Value Distribution', title_x=0.5)

## 결측치 다루기(참고)
결측치 처리를 위한 참고 자료 : 
  * [Missing Values](https://www.kaggle.com/code/alexisbcook/missing-values/tutorial) by [Alexis Cook](https://www.kaggle.com/alexisbcook)
  * [Data Cleaning Challenge:Handling missing values](https://www.kaggle.com/code/rtatman/data-cleaning-challenge-handling-missing-values/notebook) by [Rachel Tatman](https://www.kaggle.com/rtatman)
  * [A Guide to Handling Missing values in Python](https://www.kaggle.com/code/parulpandey/a-guide-to-handling-missing-values-in-python/notebook) by [Parul Pandey](https://www.kaggle.com/parulpandey)

  
누락된 값을 처리하는 기능이 있는 모델은 다음이 있다.
 * [Xgboost](https://xgboost.readthedocs.io/en/latest/faq.html)
 * [LightGBM](https://lightgbm.readthedocs.io/en/latest/Advanced-Topics.html)
 * [Catboost](https://catboost.ai/docs/concepts/algorithm-missing-values-processing.html)


## 연속형(Continuos), 범주형(Categorical) 자료 분포
 * 총 12개의 변수중에서 6개는 연속형, 2개는 텍스트 데이터, 4개는 범주형 자료이다. 
 * **HomePlanet**과 **Destination** 변수는 3개의 다른 고유한 값으로 구성되어 있다.
 * **CryoSleep**과 **VIP**은 bool 변수이다.(이산형) 

In [ ]:
df=pd.concat([train[Features], test[Features]], axis=0)
text_features=['Cabin','Name']

# 이산형 변수
cat_features = [col for col in Features if df[col].nunique() < 25 and col not in text_features]
# 연속형 변수
cont_features = [col for col in Features if df[col].nunique() >=25 and col not in text_features]

del df

In [ ]:
print(f'\033[94mTotal number of features: {len(Features)}')
print(f'\033[94mNumber of categorical features: {len(cat_features)}')
print(f'\033[94mNumber of continuos features: {len(cont_features)}')
print(f'\033[94mNumber of text features: {len(text_features)}')

labels=['Categorical','Continuos','Text']
values=[len(cat_features),len(cont_features), len(text_features)]
colors=['#DE3163','#58D68D']

fig=go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0.1, 0,0],
                          marker=dict(colors=colors,
                                     line=dict(color='#000000',
                                              width=2)))])

fig.show()

## 연속형 변수의 분포

### Age

In [ ]:
train_age=train.copy()
test_age=test.copy()

train_age['type']='Train'
test_age['type']='Test'

ageDF=pd.concat([train_age,test_age])
fig=px.histogram(data_frame=ageDF,
                x='Age', color='type', color_discrete_sequence=['#58D68D','#DE3136'],
                marginal='box',nbins=100, template='plotly_white')

fig.update_layout(title='Distribution of Age', title_x=0.5)
fig.show()

## 범주형 변수 분포

In [ ]:
if len(cat_features) == 0:
    print('No Categorical Features')
else:
    ncols=2
    nrows=2
    
    fig,axes = plt.subplots(nrows, ncols, figsize=(18,10))
    for r in range(nrows):
        for c in range(ncols):
            col=cat_features[r*ncols+c]
            sns.countplot(train[col], ax=axes[r,c], palette='viridis', label='Train data')
            sns.countplot(test[col], ax=axes[r,c], palette='magma', label='Test data')
            
            axes[r,c].legend()
            axes[r,c].set_ylabel('')
            axes[r,c].set_xlabel(col, fontsize=20)
            axes[r,c].tick_params(labelsize=10, width=.5)
            axes[r,c].xaxis.offsetText.set_fontsize(4)
            axes[r,c].yaxis.offsetText.set_fontsize(4)
            
    plt.show()

## Target 분포
* 0과 1 두 개의 target value로 구성.
* 두 값은 비슷한 분포를 나타낸다.

In [ ]:
target_df = pd.DataFrame(train[Target].value_counts()).reset_index()
target_df.columns=[Target, 'count']

fig=px.bar(data_frame=target_df, x=Target, y='count')
fig.update_traces(marker_color=['#58D68D','#DE3163'],
                 marker_line_color='rgb(0,0,0)',
                 marker_line_width=2)

fig.update_layout(title='Target Distribuion', template='plotly_white', title_x=0.5)


print(f'\033[94m Percentage of Transported = 0 : {target_df["count"][0]*100 / train.shape[0]:.2f}%')
print(f'\033[94m Percentage of Transported = 1 : {target_df["count"][1]*100 / train.shape[0]:.2f}%')

fig.show()

## 상관관계도 (Correlation Matrix)

In [ ]:
fig = px.imshow(train.corr(), text_auto=True, aspect='auto', color_continuous_scale='viridis')
fig.show()

# 데이터 전처리
 - Data Pre-Processing

## Imputing Missing Valued

In [ ]:
imputer_cols = ['Age','FoodCourt','ShoppingMall','Spa','VRDeck','RoomService']
imputer = SimpleImputer(strategy=Strategy)
imputer.fit(train[imputer_cols])

train[imputer_cols]=imputer.transform(train[imputer_cols])
test[imputer_cols]=imputer.transform(test[imputer_cols])

train['HomePlanet'].fillna('Z', inplace=True)
test['HomePlanet'].fillna('Z', inplace=True)

## 범주형 변수 인코딩

In [ ]:
label_cols=['HomePlanet', 'CryoSleep','Cabin','Destination','VIP']

def label_encoder(train,test,columns):
    for col in columns:
        train[col]=train[col].astype(str)
        test[col]=test[col].astype(str)
        train[col]=LabelEncoder().fit_transform(train[col])
        test[col]=LabelEncoder().fit_transform(test[col])
    
    return train,test

train, test = label_encoder(train,test,label_cols)

In [ ]:
train.drop(['Name','Cabin'], axis=1, inplace=True)
test.drop(['Name','Cabin'], axis=1, inplace=True)

X=train.drop(Target, axis=1)
y=train[Target]

X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=12, test_size=0.33)

# Modeing

## 27 Differnt Classifier(Lazy Predcit)

In [ ]:
clf = LazyClassifier(verbose=0, 
                    ignore_warnings=True,
                    custom_metric=None,
                    predictions=False,
                    random_state=12)

models, predictions = clf.fit(X_train, X_test, y_train, y_test)

### 상위 15개 모델

In [ ]:
models[:15]

### 결과 시각화 

In [ ]:
line=px.line(data_frame=models, y=['Accuracy'], markers=True)

line.update_xaxes(title='Model', rangeslider_visible=False)
line.update_yaxes(title='Accuracy')
line.update_traces(line_color='red')
line.update_layout(showlegend=True,
                  title={
                      'text':'Accuracy vc Model',
                      'y':0.94,
                      'x':0.5,
                      'xanchor':'center',
                      'yanchor':'top'
                  })

line.show()

In [ ]:
line = px.line(data_frame=models, y=['ROC AUC', 'F1 Score'], markers=True)

line.update_xaxes(title='Model', rangeslider_visible=False)
line.update_yaxes(title='ROC AUC SCORE')
line.update_layout(showlegend=True,
                  title={
                      'text':'ROC AUC and F1 Score vs Model',
                      'y':0.94,
                      'x':0.5,
                      'xanchor':'center',
                      'yanchor':'top'
                  })
line.show()

In [ ]:
line = px.line(data_frame=models, y=['Time Taken'], markers=True)

line.update_xaxes(title='Model', rangeslider_visible=False)
line.update_yaxes(title='Time(s)')
line.update_traces(line_color='purple')
line.update_layout(showlegend=True,
                  title={
                      'text':'Time Taken VS Model',
                      'y':0.94,
                      'x':0.5,
                      'xanchor':'center',
                      'yanchor':'top'
                  })
line.show()

## LGBM Classifier (5 Folds)

In [ ]:
lgb_params={
    'objective':'binary',
    'n_estimators':50,
    'learning_rate':0.08
}

lgb_predictions=0
lgb_scores=[]
lgb_fimp=[]

LGBM_FEATURES=list(train.columns)[:-1]
skf=StratifiedKFold(n_splits=Folds, shuffle=True, random_state=Random_state)

for fold, (train_idx, valid_idx) in enumerate(skf.split(train[LGBM_FEATURES], train[Target])):
    print(f'\033[94m')
    print(10*"=", f"FOLD={fold+1}", 10*"=")
    start_time = time.time()
    
    X_train, X_valid = train.iloc[train_idx][LGBM_FEATURES], train.iloc[valid_idx][LGBM_FEATURES]
    y_train, y_valid = train[Target].iloc[train_idx], train[Target].iloc[valid_idx]
    
    model=LGBMClassifier(**lgb_params)
    model.fit(X_train, y_train, verbose=0)
    
    preds_valid = model.predict(X_valid)
    acc=accuracy_score(y_valid, preds_valid)
    lgb_scores.append(acc)
    run_time=time.time() - start_time
    
    fim = pd.DataFrame(index=LGBM_FEATURES, data=model.feature_importances_, columns=[f'{fold}_importance'])
    lgb_fimp.append(fim)
    
    print(f"Fold={fold+1}, Accuracy score:{acc:.2f}%, Run Time:{run_time:.2f}s")
    test_preds=model.predict(test[LGBM_FEATURES])
    lgb_predictions += test_preds/Folds

print("")
print(f"Mean Accuracy : {np.mean(lgb_scores)}")

### Feature Importance

In [ ]:
lgbm_fis_df=pd.concat(lgb_fimp, axis=1).head(15)
lgbm_fis_df.sort_values('1_importance').plot(kind='barh', figsize=(15,10),
                                            title='Feature Importance Across Folds')

plt.show()

# Submission

## LGMB Classifier Submission

In [ ]:
submission[Target] = lgb_predictions.astype('bool')
submission